## Cargar Dataframes


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import (
    col, lit, expr, when, split, trim, upper, substring, round,
    count, avg, sum, max, min,
    datediff, current_date,
    array, explode, struct, regexp_replace
)
import re

In [2]:
# ==============================
# 1. Inicializar Spark Session
# ==============================
spark = SparkSession.builder \
    .appName("HDFS_NiFi_Data_Cleaning") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "4") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.host", "jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

print("Sesión creada correctamente")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 12:58:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Sesión creada correctamente


In [3]:
# ==============================
# 2. Paths en HDFS
# ==============================
hdfs_base_path = "hdfs://namenode:9000/user/nifi/"

# --- CSVs ---
hdfs_path_maestra = hdfs_base_path + "maestra.csv"
hdfs_path_zona = hdfs_base_path + "bd_zona.csv"
hdfs_path_json = hdfs_base_path + "datos.json"
hdfs_path_val_prod = hdfs_base_path + "Valores-Venta_Producto.csv"
hdfs_path_uni_prod = hdfs_base_path + "Unidades-Venta_Producto.csv"
hdfs_path_val_fam = hdfs_base_path + "Valores-Venta_Familia.csv"
hdfs_path_uni_fam = hdfs_base_path + "Unidades-Venta_Familia.csv"

In [4]:
# ===============================================
# 2. Carga de Datos en DataFrames de Spark
# ===============================================

# Opción de codificación para tildes y ñ
encoding_python = "UTF-8"

print("--- 1. Cargando Archivos Maestros (Separador ';') ---")
# Primer CSV (Maestra)
print("--- CSV Maestra ---")
df_maestra = spark.read.option("encoding", encoding_python) \
                       .csv(hdfs_path_maestra, header=True, inferSchema=True, sep=";")
df_maestra.show()

# Segundo CSV (Zona)
print("--- CSV Zona ---")
df_zona = spark.read.option("encoding", encoding_python) \
                    .csv(hdfs_path_zona, header=True, inferSchema=True, sep=";")
df_zona.show()


print("--- 2. Cargando JSON de Base de Datos ---")
print("--- Json Zona ---")
df_datos_zona_json = spark.read.json(hdfs_path_json)
df_datos_zona_json.show()


print("--- 3. Cargando CSVs de Ventas (Separador ',') ---")

# Valores Producto
print("--- CSV Valores Producto ---")
df_valores_producto = spark.read.option("encoding", encoding_python) \
                            .csv(hdfs_path_val_prod, header=True, inferSchema=True, sep=",")
df_valores_producto.show()

# Unidades Producto
print("--- CSV Unidades Producto ---")
df_unidades_producto = spark.read.option("encoding", encoding_python) \
                             .csv(hdfs_path_uni_prod, header=True, inferSchema=True, sep=",")
df_unidades_producto.show()

# Valores Familia
print("--- CSV Valores Familia ---")
df_valores_familia = spark.read.option("encoding", encoding_python) \
                           .csv(hdfs_path_val_fam, header=True, inferSchema=True, sep=",")
df_valores_familia.show()

# Unidades Familia
print("--- CSV Unidades Familia ---")
df_unidades_familia = spark.read.option("encoding", encoding_python) \
                            .csv(hdfs_path_uni_fam, header=True, inferSchema=True, sep=",")
df_unidades_familia.show()

print("--- ✅ Carga completada. Los 6 DataFrames estan listos. ---")

--- 1. Cargando Archivos Maestros (Separador ';') ---
--- CSV Maestra ---


+-----------+------------------+--------------------+
|   Producto|Numero de articulo|         Descripcion|
+-----------+------------------+--------------------+
|LAGRICEL PF|             41582|LAGRICEL OFTENO L...|
| ELIPTIC PF|             41561|ELIPTIC OFTENO 5M...|
|   LAGRICEL|             40515|LAGRICEL OFTENO 0...|
|FLUMETOL NF|             40513|FLUMETOL NF OFTEN...|
| TRAZIDEX O|             40341|TRAZIDEX OFTENO 5...|
| TRAZIDEX U|             40342|TRAZIDEX UNGENA 3...|
|  SOPHIPREN|             40338|SOPHIPREN OFTENO ...|
|       GAAP|             40498|    GAAP OFTENO 3 ML|
|   AQUADRAN|             41945|        AQUADRAN 10G|
|    GAAP PF|             41567|GAAP OFTENO LIBRE...|
|   ZEBESTEN|             41604|   ZEBESTEN 5ML PERU|
|     LANDAX|             41804|          LANDAX 5ML|
|     ELAR-B|             42098|         ELAR-B 5 ML|
|   DUSTALOX|             41121|        DUSTALOX 5ML|
|    ELIPTIC|             41076|  ELIPTIC OFTENO 5ML|
|     AGGLAD|             40

25/11/18 12:59:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-----------+------------------+------------------+------------------+---------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+-----------------+------------------+------------------+-------------------+-------------+------------------+------------------+---------+-----------------+------------------+------------------+-------------+--------------+------------------+------------------+----------+----------------+------------------+------------------+------------+----------------+------------------+------------------+------------+------------------+------------------+------------------+------------------+
|       Zona|   Producto|     Enero 

+-----------+-----------+-------------+-----------+------------+---------+---------------+------------------+--------------+------------------+-------------+------------------+------------+-------------------+-------------+------------------+------------+-------------------+------------+------------------+-----------+--------------------+------------+------------------+-----------+--------------------+------------+------------------+-----------+-------------------+-------------+------------------+------------+---------+-----------------+------------------+----------------+-------------+--------------+------------------+-------------+----------+----------------+------------------+---------------+------------+----------------+------------------+---------------+------------+--------------+------------------+-------------+------------------+
|       Zona|   Producto|Enero _Venta |Enero _TGT |Enero _PY 24|Enero _% |Febrero _Venta |     Febrero _TGT |Febrero _PY 24|       Febrero _% |Marzo _

### Exploración de datos

In [5]:
df_maestra.printSchema()
df_maestra.show(truncate=False)

root
 |-- Producto: string (nullable = true)
 |-- Numero de articulo: integer (nullable = true)
 |-- Descripcion: string (nullable = true)

+-----------+------------------+-------------------------------------------------+
|Producto   |Numero de articulo|Descripcion                                      |
+-----------+------------------+-------------------------------------------------+
|LAGRICEL PF|41582             |LAGRICEL OFTENO LIBRE DE CONSERVADORES (PF) 10 ML|
|ELIPTIC PF |41561             |ELIPTIC OFTENO 5ML PF PERU                       |
|LAGRICEL   |40515             |LAGRICEL OFTENO 0.5 ML                           |
|FLUMETOL NF|40513             |FLUMETOL NF OFTENO 5ML                           |
|TRAZIDEX O |40341             |TRAZIDEX OFTENO 5 ML.                            |
|TRAZIDEX U |40342             |TRAZIDEX UNGENA 3.5 G                            |
|SOPHIPREN  |40338             |SOPHIPREN OFTENO 5 ML                            |
|GAAP       |40498            

In [6]:
df_zona.printSchema()
df_zona.show(truncate=False)

root
 |-- Vendedor: string (nullable = true)
 |-- Nombre Cliente: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- MES NUM: integer (nullable = true)
 |-- Mes: string (nullable = true)
 |-- 2025: string (nullable = true)
 |-- CANTIDAD: string (nullable = true)

+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Vendedor   |Nombre Cliente                                                   |Producto   |MES NUM|Mes|2025   |CANTIDAD|
+-----------+-----------------------------------------------------------------+-----------+-------+---+-------+--------+
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |AGGLAD     |1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |FLUMETOL NF|1      |ENE|0      |0       |
|Pharma - N1|ADMINISTRADORA CLINICA TRESA S.A                                 |GAAP       |1      |ENE|0      |0

In [7]:
df_datos_zona_json.printSchema()
df_datos_zona_json.show(truncate=False)

root
 |-- created_at: string (nullable = true)
 |-- is_active: boolean (nullable = true)
 |-- region: string (nullable = true)
 |-- zone_code: string (nullable = true)
 |-- zone_description: string (nullable = true)
 |-- zone_id: long (nullable = true)
 |-- zone_name: string (nullable = true)

+--------------------------+---------+--------+---------+----------------------------------------------+-------+-----------+
|created_at                |is_active|region  |zone_code|zone_description                              |zone_id|zone_name  |
+--------------------------+---------+--------+---------+----------------------------------------------+-------+-----------+
|2025-09-29 21:30:27.300355|true     |Centro  |Z1       |Zona de Lima Metropolitana                    |1      |Lima       |
|2025-09-29 21:30:27.300355|true     |Centro  |Z2       |Zona de Callao y Provincia Constitucional     |2      |Callao     |
|2025-09-29 21:30:27.300355|true     |Norte   |Z3       |Zona de Norte Chico - H

In [8]:
df_valores_producto.printSchema()
df_valores_producto.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : double (nullable = true)
 |-- Enero _TGT : double (nullable = true)
 |-- Enero _PY 24: double (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : double (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: double (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : double (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: double (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : double (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: double (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : double (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: double (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : double (nullable = true)
 |-- 

In [9]:
df_unidades_producto.printSchema()
df_unidades_producto.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : integer (nullable = true)
 |-- Enero _TGT : integer (nullable = true)
 |-- Enero _PY 24: integer (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : integer (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: integer (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : integer (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: integer (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : integer (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: integer (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : integer (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: integer (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : integer (nullable =

In [10]:
df_valores_familia.printSchema()
df_valores_familia.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : double (nullable = true)
 |-- Enero _TGT : double (nullable = true)
 |-- Enero _PY 24: double (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : double (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: double (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : double (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: double (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : double (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: double (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : double (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: double (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : double (nullable = true)
 |-- 

In [11]:
df_unidades_familia.printSchema()
df_unidades_familia.show(truncate=False)

root
 |-- Zona: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- Enero _Venta : integer (nullable = true)
 |-- Enero _TGT : integer (nullable = true)
 |-- Enero _PY 24: integer (nullable = true)
 |-- Enero _% : integer (nullable = true)
 |-- Febrero _Venta : integer (nullable = true)
 |-- Febrero _TGT : double (nullable = true)
 |-- Febrero _PY 24: integer (nullable = true)
 |-- Febrero _% : double (nullable = true)
 |-- Marzo _Venta : integer (nullable = true)
 |-- Marzo _TGT : double (nullable = true)
 |-- Marzo _PY 24: integer (nullable = true)
 |-- Marzo _% : double (nullable = true)
 |-- Abril _Venta : integer (nullable = true)
 |-- Abril _TGT : double (nullable = true)
 |-- Abril _PY 24: integer (nullable = true)
 |-- Abril _% : double (nullable = true)
 |-- Mayo _Venta : integer (nullable = true)
 |-- Mayo _TGT : double (nullable = true)
 |-- Mayo _PY 24: integer (nullable = true)
 |-- Mayo _% : double (nullable = true)
 |-- Junio_Venta : integer (nullable =

## TRANSFORMACIÓN

## Paso 1: Transformación de la Tabla Transaccional (Ventas Cliente)

En este paso normalizamos la tabla principal bd_zona.csv. Renombramos columnas confusas, ajustamos los tipos de datos (números y texto) y extraemos el código de la zona para poder cruzar información más adelante.

In [12]:
print("--- 🚀 INICIANDO PROCESO ELT PARA MODELO DE RECOMENDACIÓN ---")
# ==============================================================================
# PASO 1: LIMPIEZA DE LA TABLA PRINCIPAL (df_zona)
# ==============================================================================

# Renombramos columnas para estándares de ingeniería de datos
df_transaccional = df_zona.withColumnRenamed("2025", "Venta_Valor") \
                          .withColumnRenamed("CANTIDAD", "Venta_Unidades") \
                          .withColumnRenamed("MES NUM", "Mes_Num") \
                          .withColumnRenamed("Nombre Cliente", "Cliente")

# Extraemos el código de zona (ej. de "Pharma - Z1" obtenemos "Z1")
df_transaccional = df_transaccional.withColumn(
    "zone_code_join", 
    trim(split(col("Vendedor"), "-").getItem(1))
)

# Aseguramos que los montos y cantidades sean numéricos
df_transaccional = df_transaccional.withColumn("Venta_Valor", col("Venta_Valor").cast(DoubleType())) \
                                   .withColumn("Venta_Unidades", col("Venta_Unidades").cast(IntegerType())) \
                                   .withColumn("Mes_Num", col("Mes_Num").cast(IntegerType()))

--- 🚀 INICIANDO PROCESO ELT PARA MODELO DE RECOMENDACIÓN ---


## Paso 2: Preparación de Dimensiones (Geografía y Producto)

Preparamos las tablas maestras. Del JSON geográfico seleccionamos solo lo necesario (Región, Zona). De la Maestra de Productos, obtenemos el nombre oficial para estandarizar las descripciones.

In [13]:
# ==============================================================================
# PASO 2: PREPARACIÓN DE DIMENSIONES (GEO Y PRODUCTO)
# ==============================================================================

# 2a. Dimensión Geográfica (JSON)
df_geo = df_datos_zona_json.select(
    trim(col("zone_code")).alias("zone_code"),
    col("zone_name").alias("Nombre_Zona"),
    col("region").alias("Region")
)

# 2b. Dimensión Producto (CSV Maestra)
# Seleccionamos la llave (Producto) y el nombre real (Descripcion)
df_prod_maestra = df_maestra.select(
    trim(col("Producto")).alias("Producto_Key"), 
    col("Descripcion").alias("Nombre_Producto_Oficial"),
    col("Numero de articulo").alias("ID_Articulo") 
)

## Paso 3: Enriquecimiento Inicial y Cálculo de Precios

Unimos las ventas con la geografía y los nombres oficiales. Además, creamos una variable clave para el modelo: el Precio Unitario, que nos ayudará a entender la elasticidad de la demanda.

In [14]:
# ==============================================================================
# PASO 3: ENRIQUECIMIENTO INICIAL (JOINS) Y FEATURE ENGINEERING BÁSICO
# ==============================================================================

# Join 1: Ventas + Geografía
df_step1 = df_transaccional.join(
    df_geo,
    df_transaccional.zone_code_join == df_geo.zone_code,
    "left"
)

# Join 2: Resultado + Maestra de Productos
df_master_analytics = df_step1.join(
    df_prod_maestra,
    trim(df_step1.Producto) == df_prod_maestra.Producto_Key,
    "left"
)

# Selección final y Cálculo de Precio Unitario
# IMPORTANTE: Aquí agregamos 'zone_code_join' y 'Producto_Key' para usarlas después
df_final = df_master_analytics.withColumn(
    "Precio_Unitario", 
    when(col("Venta_Unidades") > 0, 
         round(col("Venta_Valor") / col("Venta_Unidades"), 2)
    ).otherwise(0.0)
).select(
    col("zone_code_join"), # <--- Necesario para cruzar con Contexto Zona
    col("Producto_Key"),   # <--- Necesario para cruzar con Contexto Producto
    "Region",
    "Nombre_Zona",
    "Cliente",
    # Si existe nombre oficial lo usa, si no, usa el original
    when(col("Nombre_Producto_Oficial").isNotNull(), col("Nombre_Producto_Oficial"))
      .otherwise(col("Producto")).alias("Producto"), 
    "ID_Articulo",
    "Mes_Num",
    "Mes",
    "Venta_Unidades",
    "Venta_Valor",
    "Precio_Unitario"
)

print("--- ✅ Fase 1 Completada: Dataset Transaccional con Llaves ---")
df_final.printSchema() # Verifica que zone_code_join y Producto_Key aparezcan aquí

--- ✅ Fase 1 Completada: Dataset Transaccional con Llaves ---
root
 |-- zone_code_join: string (nullable = true)
 |-- Producto_Key: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Nombre_Zona: string (nullable = true)
 |-- Cliente: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- ID_Articulo: integer (nullable = true)
 |-- Mes_Num: integer (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Venta_Unidades: integer (nullable = true)
 |-- Venta_Valor: double (nullable = true)
 |-- Precio_Unitario: double (nullable = true)



In [15]:
df_final.show()

+--------------+------------+------+-----------+--------------------+--------------------+-----------+-------+---+--------------+-----------+---------------+
|zone_code_join|Producto_Key|Region|Nombre_Zona|             Cliente|            Producto|ID_Articulo|Mes_Num|Mes|Venta_Unidades|Venta_Valor|Precio_Unitario|
+--------------+------------+------+-----------+--------------------+--------------------+-----------+-------+---+--------------+-----------+---------------+
|            N1|      AGGLAD| Norte|   Chiclayo|ADMINISTRADORA CL...|  AGGLAD OFTENO 5 ML|      40351|      1|ENE|             0|        0.0|            0.0|
|            N1| FLUMETOL NF| Norte|   Chiclayo|ADMINISTRADORA CL...|FLUMETOL NF OFTEN...|      40513|      1|ENE|             0|        0.0|            0.0|
|            N1|        GAAP| Norte|   Chiclayo|ADMINISTRADORA CL...|    GAAP OFTENO 3 ML|      40498|      1|ENE|             0|        0.0|            0.0|
|            N1|    LAGRICEL| Norte|   Chiclayo|ADMI

## Paso 4: Integración del Contexto de Mercado (Unidades y Valores)

Aquí transformamos los archivos de reporte (que vienen con meses en columnas) a formato filas usando stack. Esto agrega al dataset las Metas y la Venta Histórica (PY) de la zona, permitiendo al modelo comparar el desempeño del cliente contra el mercado.

In [16]:
# ==============================================================================
# PASO 4: ENRIQUECIMIENTO CON CONTEXTO DE MERCADO (PRODUCTO)
# ==============================================================================
print("--- Procesando Contexto de Mercado (Unidades y Valores por Zona) ---")

# --- 🛠️ SUB-RUTINA DE CORRECCIÓN DE TIPOS 🛠️ ---
# Esta función fuerza a que todas las columnas numéricas sean Double
# para evitar el error "DATATYPE_MISMATCH" en el stack.
def cast_metrics_to_double(df):
    cols_to_cast = [c for c in df.columns if "TGT" in c or "PY" in c]
    for column_name in cols_to_cast:
        df = df.withColumn(column_name, col(column_name).cast(DoubleType()))
    return df

# 1. Aplicamos la corrección a los DataFrames de entrada
df_unidades_producto = cast_metrics_to_double(df_unidades_producto)
df_valores_producto = cast_metrics_to_double(df_valores_producto)
# También aplicamos a los de familia de una vez (para el Bloque 6)
df_unidades_familia = cast_metrics_to_double(df_unidades_familia)
df_valores_familia = cast_metrics_to_double(df_valores_familia)


# --- CONTINUAMOS CON EL PROCESO NORMAL ---

# Función auxiliar para la expresión 'stack'
def get_stack_expr(metric_suffix):
    return f"""stack(12, 
        'ENE', `Enero _{metric_suffix}`, `Enero _PY 24`,
        'FEB', `Febrero _{metric_suffix}`, `Febrero _PY 24`,
        'MAR', `Marzo _{metric_suffix}`, `Marzo _PY 24`,
        'ABR', `Abril _{metric_suffix}`, `Abril _PY 24`,
        'MAY', `Mayo _{metric_suffix}`, `Mayo _PY 24`,
        'JUN', `Junio_{metric_suffix}`, `Junio_PY 24`,
        'JUL', `Julio_{metric_suffix}`, `Julio_PY 24`,
        'AGO', `Agosto_{metric_suffix}`, `Agosto_PY 24`,
        'SEP', `Septiembre_{metric_suffix}`, `Septiembre_PY 24`,
        'OCT', `Octubre_{metric_suffix}`, `Octubre_PY 24`,
        'NOV', `Noviembre_{metric_suffix}`, `Noviembre_PY 24`,
        'DIC', `Diciembre_{metric_suffix}`, `Diciembre_PY 24`
    )"""

# 4a. Procesar Unidades (TGT = Meta)
df_ctx_unidades = df_unidades_producto.select(
    col("Zona"), col("Producto"),
    expr(get_stack_expr("TGT ") + " as (Mes_Corto, Meta_Zona_Unidades, Venta_PY_Zona_Unidades)")
)

# 4b. Procesar Valores (Dinero)
df_ctx_valores = df_valores_producto.select(
    col("Zona"), col("Producto"),
    expr(get_stack_expr("TGT ") + " as (Mes_Corto, Meta_Zona_Valor, Venta_PY_Zona_Valor)")
)

# 4c. Unir ambos contextos (Unidades + Valores)
df_contexto_producto = df_ctx_unidades.join(
    df_ctx_valores, 
    on=["Zona", "Producto", "Mes_Corto"], 
    how="inner"
)

# Limpieza de llaves para el Join
df_contexto_producto = df_contexto_producto.withColumn("zone_code_join", trim(split(col("Zona"), "-").getItem(1))) \
                                           .withColumn("Producto_Key", trim(col("Producto"))) \
                                           .withColumn("Mes_Corto", upper(col("Mes_Corto")))

# 4d. JOIN FINAL al Dataset Principal
df_dataset_completo = df_final.join(
    df_contexto_producto,
    (df_final.zone_code_join == df_contexto_producto.zone_code_join) & 
    (df_final.Producto_Key == df_contexto_producto.Producto_Key) & 
    (df_final.Mes == df_contexto_producto.Mes_Corto),
    "left"
).select(
    df_final["*"],
    col("Meta_Zona_Unidades"), col("Venta_PY_Zona_Unidades"),
    col("Meta_Zona_Valor"), col("Venta_PY_Zona_Valor")
).na.fill(0, subset=["Meta_Zona_Unidades", "Venta_PY_Zona_Unidades", "Meta_Zona_Valor", "Venta_PY_Zona_Valor"])

# Calculamos Precio Promedio de la Zona (Feature Engineering Avanzado)
df_dataset_completo = df_dataset_completo.withColumn(
    "Precio_Promedio_PY_Zona",
    when(col("Venta_PY_Zona_Unidades") > 0, 
         round(col("Venta_PY_Zona_Valor") / col("Venta_PY_Zona_Unidades"), 2)
    ).otherwise(0.0)
)

print("--- Contexto de Producto Integrado Correctamente ---")

--- Procesando Contexto de Mercado (Unidades y Valores por Zona) ---
--- Contexto de Producto Integrado Correctamente ---


In [17]:
df_dataset_completo.show()

+--------------+------------+--------+-----------+--------------------+--------------------+-----------+-------+---+--------------+-----------+---------------+------------------+----------------------+-----------------+-------------------+-----------------------+
|zone_code_join|Producto_Key|  Region|Nombre_Zona|             Cliente|            Producto|ID_Articulo|Mes_Num|Mes|Venta_Unidades|Venta_Valor|Precio_Unitario|Meta_Zona_Unidades|Venta_PY_Zona_Unidades|  Meta_Zona_Valor|Venta_PY_Zona_Valor|Precio_Promedio_PY_Zona|
+--------------+------------+--------+-----------+--------------------+--------------------+-----------+-------+---+--------------+-----------+---------------+------------------+----------------------+-----------------+-------------------+-----------------------+
|             S|  ELIPTIC PF|Nacional|    Cadenas| ANGELES DE LA SALUD|ELIPTIC OFTENO 5M...|      41561|      1|ENE|             5|     169.83|          33.97|             312.0|                   5.0|       

## Paso 5: Integración de Jerarquías (Familias)

Finalmente, añadimos el nivel más alto de abstracción. Usamos la lógica de la "Primera Palabra" (ej. asociar "ELIPTIC PF" con la familia "ELIPTIC") para traer tendencias generales, vital para predecir sobre productos nuevos.

In [18]:
# ==============================================================================
# PASO 5: INTEGRACIÓN DE FAMILIAS (JERARQUÍA)
# ==============================================================================
print("--- Iniciando Integración de Contexto Familiar (Jerarquías) ---")

# 5a. Procesar Unidades y Valores de Familia (Unpivot)
df_fam_uni = df_unidades_familia.select(
    col("Zona"), col("Producto").alias("Familia_Nom"),
    expr(get_stack_expr("TGT ") + " as (Mes_Corto, Meta_Fam_Uni, Venta_PY_Fam_Uni)")
)

df_fam_val = df_valores_familia.select(
    col("Zona"), col("Producto").alias("Familia_Nom"),
    expr(get_stack_expr("TGT ") + " as (Mes_Corto, Meta_Fam_Val, Venta_PY_Fam_Val)")
)

# 5b. Unir contexto familia
df_familia_master = df_fam_uni.join(df_fam_val, on=["Zona", "Familia_Nom", "Mes_Corto"], how="outer")

# 5c. Crear la "Llave Mágica" (Primera Palabra)
# Extraemos la primera palabra de la Familia (ej. "ELIPTIC" de "ELIPTIC FAM")
df_familia_master = df_familia_master.withColumn(
    "Family_Join_Key", trim(split(col("Familia_Nom"), " ").getItem(0))
).withColumn(
    "zone_code_join", trim(split(col("Zona"), "-").getItem(1))
).withColumn("Mes_Corto", upper(col("Mes_Corto")))

# Extraemos la primera palabra del Producto en el dataset principal
df_dataset_completo = df_dataset_completo.withColumn(
    "Family_Join_Key", trim(split(col("Producto"), " ").getItem(0))
)

# 5d. JOIN FINAL: Dataset Completo + Familia
df_ml_final_v2 = df_dataset_completo.join(
    df_familia_master,
    (df_dataset_completo.zone_code_join == df_familia_master.zone_code_join) &
    (df_dataset_completo.Family_Join_Key == df_familia_master.Family_Join_Key) &
    (df_dataset_completo.Mes == df_familia_master.Mes_Corto),
    "left"
).select(
    df_dataset_completo["*"],
    col("Meta_Fam_Uni").alias("Meta_Familia_Unidades"),
    col("Venta_PY_Fam_Uni").alias("Venta_PY_Familia_Unidades"),
    col("Meta_Fam_Val").alias("Meta_Familia_Valor"),
    col("Venta_PY_Fam_Val").alias("Venta_PY_Familia_Valor")
).na.fill(0)

# Limpieza final de columnas auxiliares
df_ml_final_v2 = df_ml_final_v2.drop("zone_code_join", "Mes_Corto", "Family_Join_Key", "Producto_Key")

--- Iniciando Integración de Contexto Familiar (Jerarquías) ---


## Paso 6: Visualización Final

Verificamos que todo esté correcto. Este DataFrame df_ml_final_v2 es el activo final que entrará a tu algoritmo Random Forest.

In [19]:
# ==============================================================================
# RESULTADO FINAL
# ==============================================================================
print("--- ✅ Dataset FINAL Completado (Clientes + Productos + Familias) ---")
df_ml_final_v2.printSchema()

print("--- Muestra de datos enriquecidos ---")
df_ml_final_v2.select(
    "Cliente", "Producto", "Mes", 
    "Venta_Unidades", "Precio_Unitario", 
    "Precio_Promedio_PY_Zona", "Venta_PY_Familia_Unidades"
).show(10, truncate=False)

# Comando para guardar (Descomentar cuando estés listo para guardar en HDFS)
# df_ml_final_v2.write.mode("overwrite").parquet(hdfs_base_path + "processed/dataset_ml_sophia_final")

--- ✅ Dataset FINAL Completado (Clientes + Productos + Familias) ---
root
 |-- Region: string (nullable = true)
 |-- Nombre_Zona: string (nullable = true)
 |-- Cliente: string (nullable = true)
 |-- Producto: string (nullable = true)
 |-- ID_Articulo: integer (nullable = true)
 |-- Mes_Num: integer (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Venta_Unidades: integer (nullable = true)
 |-- Venta_Valor: double (nullable = false)
 |-- Precio_Unitario: double (nullable = false)
 |-- Meta_Zona_Unidades: double (nullable = false)
 |-- Venta_PY_Zona_Unidades: double (nullable = false)
 |-- Meta_Zona_Valor: double (nullable = false)
 |-- Venta_PY_Zona_Valor: double (nullable = false)
 |-- Precio_Promedio_PY_Zona: double (nullable = false)
 |-- Meta_Familia_Unidades: double (nullable = false)
 |-- Venta_PY_Familia_Unidades: double (nullable = false)
 |-- Meta_Familia_Valor: double (nullable = false)
 |-- Venta_PY_Familia_Valor: double (nullable = false)

--- Muestra de datos enriqu

In [20]:
df_ml_final_v2.show(truncate=False)

+--------+-----------+-----------------------------------------------------------------+-------------------------------------------------+-----------+-------+---+--------------+-----------+---------------+------------------+----------------------+-----------------+-------------------+-----------------------+---------------------+-------------------------+------------------+----------------------+
|Region  |Nombre_Zona|Cliente                                                          |Producto                                         |ID_Articulo|Mes_Num|Mes|Venta_Unidades|Venta_Valor|Precio_Unitario|Meta_Zona_Unidades|Venta_PY_Zona_Unidades|Meta_Zona_Valor  |Venta_PY_Zona_Valor|Precio_Promedio_PY_Zona|Meta_Familia_Unidades|Venta_PY_Familia_Unidades|Meta_Familia_Valor|Venta_PY_Familia_Valor|
+--------+-----------+-----------------------------------------------------------------+-------------------------------------------------+-----------+-------+---+--------------+-----------+-----------

## Paso 7: Validación y limpieza final para ML

In [21]:
print("--- 🧹 Ejecutando Limpieza Final Pre-Modelo ---")

# 1. Eliminar registros donde la variable objetivo sea nula o negativa (si aplica)
df_ready = df_ml_final_v2.na.fill(0, subset=[
    "Venta_Unidades", "Venta_Valor", "Precio_Unitario",
    "Meta_Zona_Unidades", "Venta_PY_Zona_Unidades",
    "Precio_Promedio_PY_Zona"
])

# 2. Verificar si quedan Nulos críticos
# Random Forest en Spark NO acepta nulos en las columnas de entrada (features)
from pyspark.sql.functions import col, sum as _sum

print("--- Chequeo de Nulos Restantes (Debería ser todo 0) ---")
df_ready.select([
    _sum(col(c).isNull().cast("int")).alias(c) 
    for c in df_ready.columns
]).show()

# 3. Filtrar inconsistencias extremas (Outliers obvios)
# Ejemplo: Precio Unitario negativo o infinito
df_ready = df_ready.filter(col("Precio_Unitario") >= 0)

print(f"--- Dataset Listo. Total Filas: {df_ready.count()} ---")

--- 🧹 Ejecutando Limpieza Final Pre-Modelo ---
--- Chequeo de Nulos Restantes (Debería ser todo 0) ---
+------+-----------+-------+--------+-----------+-------+---+--------------+-----------+---------------+------------------+----------------------+---------------+-------------------+-----------------------+---------------------+-------------------------+------------------+----------------------+
|Region|Nombre_Zona|Cliente|Producto|ID_Articulo|Mes_Num|Mes|Venta_Unidades|Venta_Valor|Precio_Unitario|Meta_Zona_Unidades|Venta_PY_Zona_Unidades|Meta_Zona_Valor|Venta_PY_Zona_Valor|Precio_Promedio_PY_Zona|Meta_Familia_Unidades|Venta_PY_Familia_Unidades|Meta_Familia_Valor|Venta_PY_Familia_Valor|
+------+-----------+-------+--------+-----------+-------+---+--------------+-----------+---------------+------------------+----------------------+---------------+-------------------+-----------------------+---------------------+-------------------------+------------------+----------------------+
|     

## Paso 8: Almacenamiento

In [24]:
# ==============================================================================
# 5. ALMACENAMIENTO (DATA ENGINEERING FINALIZADO)
# ==============================================================================
print("--- 💾 Guardando dataset procesado en HDFS (Formato Parquet) ---")

# Ruta donde se guardará el archivo maestro
ruta_destino_parquet = hdfs_base_path + "processed/dataset_ml_sophia_final"

# Guardamos (mode='overwrite' reemplaza si ya existe)
df_ready.write.mode("overwrite").parquet(ruta_destino_parquet)

print(f"✅ Dataset guardado exitosamente en: {ruta_destino_parquet}")

--- 💾 Guardando dataset procesado en HDFS (Formato Parquet) ---


[Stage 145:>                                                        (0 + 1) / 1]

✅ Dataset guardado exitosamente en: hdfs://namenode:9000/user/nifi/processed/dataset_ml_sophia_final


# Entrenamiento del Modelo

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# ==============================================================================
# 6. ENTRENAMIENTO DEL MODELO (CON SPLIT: TRAIN / VALIDATION / TEST)
# ==============================================================================
print("--- 🤖 Iniciando Configuración del Modelo Random Forest ---")

# 1. Cargar Datos
path_df_ml = "hdfs://namenode:9000/user/nifi/processed/dataset_ml_sophia_final"
df_full = spark.read.parquet(path_df_ml)

# 2. Definir Variables
categorical_cols = ["Cliente", "Producto", "Nombre_Zona", "Region", "Mes"]
numerical_cols = [
    "Precio_Unitario", 
    "Mes_Num",
    "Meta_Zona_Unidades", 
    "Venta_PY_Zona_Unidades",
    "Precio_Promedio_PY_Zona",
    "Venta_PY_Familia_Unidades"
]
label_col = "Venta_Unidades"

# 3. Construir el Pipeline
stages = []

# Paso A: Indexar Textos
for col_name in categorical_cols:
    indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_Index", handleInvalid="keep")
    stages.append(indexer)

# Paso B: Vector Assembler
input_cols_assembler = [c + "_Index" for c in categorical_cols] + numerical_cols
assembler = VectorAssembler(inputCols=input_cols_assembler, outputCol="features", handleInvalid="keep")
stages.append(assembler)

# Paso C: Random Forest (Con corrección maxBins=500)
rf = RandomForestRegressor(
    featuresCol="features", 
    labelCol=label_col, 
    numTrees=50, 
    seed=42,
    maxBins=500 
)
stages.append(rf)

# ---------------------------------------------------------
# 4. SPLIT DE DATOS (70% Train - 20% Validation - 10% Test)
# ---------------------------------------------------------
print("--- ✂️ Separando datos en 3 conjuntos ---")
# Los pesos deben sumar 1.0 (0.7 + 0.2 + 0.1)
(train_data, val_data, test_data) = df_full.randomSplit([0.7, 0.2, 0.1], seed=42)

print(f"Total Registros: {df_full.count()}")
print(f" - Train (Entrenamiento): {train_data.count()} filas")
print(f" - Validation (Ajuste):   {val_data.count()} filas")
print(f" - Test (Prueba Final):   {test_data.count()} filas")

# 5. Ejecutar Entrenamiento (Solo con Train)
print("\n--- 🧠 Entrenando el modelo con TRAIN SET... ---")
pipeline = Pipeline(stages=stages)
model = pipeline.fit(train_data)

# ---------------------------------------------------------
# 6. EVALUACIÓN
# ---------------------------------------------------------
evaluator_rmse = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="mae")

# A) Evaluación en VALIDATION (Para detectar Overfitting)
print("\n--- 🔍 Evaluando en VALIDATION SET ---")
pred_val = model.transform(val_data)
rmse_val = evaluator_rmse.evaluate(pred_val)
mae_val = evaluator_mae.evaluate(pred_val)
print(f"Validation RMSE: {rmse_val:.2f}")
print(f"Validation MAE:  {mae_val:.2f}")

# B) Evaluación en TEST (Resultado Final)
print("\n--- 🏆 Evaluando en TEST SET (Prueba de Fuego) ---")
pred_test = model.transform(test_data)
rmse_test = evaluator_rmse.evaluate(pred_test)
mae_test = evaluator_mae.evaluate(pred_test)

print("="*40)
print(f"📊 RESULTADOS FINALES (TEST SET)")
print("="*40)
print(f"RMSE (Error Cuadrático Medio): {rmse_test:.2f}")
print(f"MAE (Error Absoluto Medio):    {mae_test:.2f}")
print("-" * 40)

# Análisis Rápido de Overfitting
diff = rmse_test - rmse_val
print("Diagnóstico de Salud del Modelo:")
if diff > 5: 
    print("⚠️ ALERTA: El error en Test es mucho mayor que en Validation. Posible Overfitting (Memorizó los datos).")
else:
    print("✅ SALUDABLE: El error es consistente entre Validation y Test. El modelo generaliza bien.")
print("="*40)

# Ver predicciones finales
pred_test.select("Cliente", "Producto", "Venta_Unidades", "prediction") \
         .orderBy("prediction", ascending=False) \
         .show(5, truncate=False)

--- 🤖 Iniciando Configuración del Modelo Random Forest ---
--- ✂️ Separando datos en 3 conjuntos ---
Total Registros: 4884
 - Train (Entrenamiento): 3490 filas
 - Validation (Ajuste):   914 filas
 - Test (Prueba Final):   480 filas

--- 🧠 Entrenando el modelo con TRAIN SET... ---

--- 🔍 Evaluando en VALIDATION SET ---
Validation RMSE: 81.52
Validation MAE:  38.94

--- 🏆 Evaluando en TEST SET (Prueba de Fuego) ---
📊 RESULTADOS FINALES (TEST SET)
RMSE (Error Cuadrático Medio): 106.13
MAE (Error Absoluto Medio):    46.42
----------------------------------------
Diagnóstico de Salud del Modelo:
⚠️ ALERTA: El error en Test es mucho mayor que en Validation. Posible Overfitting (Memorizó los datos).
+--------------------------------------------------------+-------------------------------------------------+--------------+-----------------+
|Cliente                                                 |Producto                                         |Venta_Unidades|prediction       |
+-------------

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col, when

# ==============================================================================
# 7. EVALUACIÓN DETALLADA: MATRIZ DE CONFUSIÓN, MÉTRICAS Y AUC
# ==============================================================================
print("--- 📊 Generando Reporte de Clasificación Avanzado ---")

# 1. Binarizar Predicciones (Convertir Regresión a Clasificación)
# Regla de Negocio: 
# - Realidad: Si vendió > 0, es un "1" (Cliente Activo).
# - Predicción: Si el modelo dice >= 1 unidad, es un "1" (Recomendación de Venta).
df_results = predictions.withColumn(
    "Label_Bin", when(col("Venta_Unidades") > 0, 1.0).otherwise(0.0)
).withColumn(
    "Pred_Bin", when(col("prediction") >= 1.0, 1.0).otherwise(0.0)
)

# 2. Calcular los 4 Cuadrantes de la Matriz de Confusión
# TP = True Positive (Predijo Compra y SÍ Compró)
# TN = True Negative (Predijo NO Compra y NO Compró)
# FP = False Positive (Predijo Compra y NO Compró) - Error Tipo I
# FN = False Negative (Predijo NO Compra y SÍ Compró) - Error Tipo II (Venta Perdida)

tp = df_results.filter((col("Pred_Bin") == 1.0) & (col("Label_Bin") == 1.0)).count()
tn = df_results.filter((col("Pred_Bin") == 0.0) & (col("Label_Bin") == 0.0)).count()
fp = df_results.filter((col("Pred_Bin") == 1.0) & (col("Label_Bin") == 0.0)).count()
fn = df_results.filter((col("Pred_Bin") == 0.0) & (col("Label_Bin") == 1.0)).count()

total = tp + tn + fp + fn

# 3. Calcular Métricas Derivadas
accuracy = (tp + tn) / total
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # ¿Qué tan fiables son mis "sí"?
recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0     # ¿Cuántos de los "sí" reales encontré?
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

# 4. Calcular AUC (Usando la predicción numérica original para mayor precisión)
binary_evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="prediction", 
    labelCol="Label_Bin", 
    metricName="areaUnderROC"
)
auc = binary_evaluator.evaluate(df_results)

# ==============================================================================
# REPORTE FINAL VISUAL
# ==============================================================================

print("\n" + "="*60)
print("🏁 MATRIZ DE CONFUSIÓN (REALIDAD vs PREDICCIÓN)")
print("="*60)
print(f"{'':<20} | {'Realidad: NO (0)':<15} | {'Realidad: SÍ (1)':<15}")
print("-" * 60)
print(f"{'Predicción: NO (0)':<20} | {tn:<15} | {fn:<15} (Falsos Negativos)")
print(f"{'Predicción: SÍ (1)':<20} | {fp:<15} | {tp:<15} (Aciertos)")
print("-" * 60)
print(f"Total Evaluados: {total}")

print("\n" + "="*60)
print("📈 MÉTRICAS DE DESEMPEÑO")
print("="*60)
print(f"1. Accuracy (Exactitud Global):   {accuracy:.2%}  (De cada 100 casos, aciertas en {accuracy*100:.0f})")
print(f"2. Precision (Calidad Recomend.): {precision:.2%}  (Si recomiendas, el {precision*100:.0f}% compra)")
print(f"3. Recall (Sensibilidad):         {recall:.2%}  (Detectas el {recall*100:.0f}% de los clientes reales)")
print(f"4. F1-Score (Balance):            {f1_score:.2%}  (Equilibrio entre Precisión y Recall)")
print("-" * 60)
print(f"🌟 AUC (Area Under Curve):        {auc:.4f}")
print("="*60)

print("\nInterpretación para tu Negocio:")
if precision < 0.5:
    print("⚠️ Cuidado: Tu Precisión es baja. Estás recomendando productos a mucha gente que no los quiere.")
elif recall < 0.5:
    print("⚠️ Cuidado: Tu Recall es bajo. Estás perdiendo muchas oportunidades de venta (clientes que compran y no detectas).")
else:
    print("✅ ¡Buen trabajo! El modelo tiene un balance saludable.")

--- 📊 Generando Reporte de Clasificación Avanzado ---

🏁 MATRIZ DE CONFUSIÓN (REALIDAD vs PREDICCIÓN)
                     | Realidad: NO (0) | Realidad: SÍ (1)
------------------------------------------------------------
Predicción: NO (0)   | 0               | 0               (Falsos Negativos)
Predicción: SÍ (1)   | 30              | 903             (Aciertos)
------------------------------------------------------------
Total Evaluados: 933

📈 MÉTRICAS DE DESEMPEÑO
1. Accuracy (Exactitud Global):   96.78%  (De cada 100 casos, aciertas en 97)
2. Precision (Calidad Recomend.): 96.78%  (Si recomiendas, el 97% compra)
3. Recall (Sensibilidad):         100.00%  (Detectas el 100% de los clientes reales)
4. F1-Score (Balance):            98.37%  (Equilibrio entre Precisión y Recall)
------------------------------------------------------------
🌟 AUC (Area Under Curve):        0.5175

Interpretación para tu Negocio:
✅ ¡Buen trabajo! El modelo tiene un balance saludable.
